# LangChain Expression Language

### What is LCEL?

* A declarative way to build AI pipelines in LangChain.
* Instead of long Python code, you connect components in a chain-like style.

---

### Why LCEL?

* Shorter and cleaner code.
* Easier to read and maintain.
* Works the same for synchronous, asynchronous, batch, and streaming execution.
* Good for debugging and reusing components.

---

### Basic Example

Without LCEL:

```python
prompt = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = LLMChain(llm=ChatOpenAI(), prompt=prompt)
result = chain.run("cats")
```

With LCEL:

```python
chain = prompt | ChatOpenAI() | StrOutputParser()
result = chain.invoke({"topic": "cats"})
```
**prompt, ChatOpenAI(), StrOutputParser() are called runnables**

---

## LCEL Operators

* `|` → Pipe operator, connects steps.
* `.invoke()` → Run once with input.
* `.batch()` → Run for multiple inputs.
* `.stream()` → Get outputs step by step (streaming responses).

---

## Key Features

1. Composable – you can plug and play different parts.
2. Parallel and sequential execution supported.
3. Unified API – works the same for sync, async, batch, and streaming.
4. Production-ready – optimized for speed and scalability.

---

## Simple Summary

* LCEL is a short and readable way to connect LangChain components.
* It works like piping data through different steps.
* Use `|` to chain steps.
* The same code works for real-time, async, or batch calls.

---


## Piping a prompt, model, and an output parser

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser
import config

In [3]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

list_instructions

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [16]:
# Chat Template

chat_template = ChatPromptTemplate.from_messages([
    ('human', "I've recently adopted a {pet}. Could you suggest three {pet} names? \n" + list_instructions)
])

print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [17]:
# Chat Model

def AskGeminiAI(question):
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash",
        google_api_key=config.gemini_api_key,
        temperature=0.7,
        max_tokens=250,
        model_kwargs={"seed": 42}
    )
    response = llm.invoke(question)
    return response.content

In [18]:
# Output parser

list_output_parser = CommaSeparatedListOutputParser()

In [28]:
# 3 step

chat_template_result = chat_template.invoke({'pet':'dog'})

chat_result = AskGeminiAI(chat_template_result)

list_output_parser.invoke(chat_result)

['Buddy', 'Luna', 'Charlie']

In [29]:
# Single line

chain = chat_template | AskGeminiAI | list_output_parser

chain.invoke({'pet':'dog'})

['Buddy', 'Luna', 'Charlie']